<a href="https://colab.research.google.com/github/iurysilva/Classificacao-com-Rede-Neural-e-arvore-de-Decisao/blob/master/Classifica%C3%A7%C3%A3o_%C3%81rvore_de_Decis%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from collections import Counter
import statistics as st
import math


In [3]:
bd_vinho = pd.read_csv('banco_vinho.csv')
bd_vinho.head()


,Vinho,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,Diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


# Ferramentas

In [ ]:
def retorna_treino(base):
    treino = base.sample(frac=0.7)
    return treino

def retorna_teste(base, base_treino):
    teste = base.drop(base_treino.index)
    return teste

def calcula_resultados(matriz, verbose=False):
    num_classes = len(matriz)
    
    sensibilidade = np.zeros([num_classes])
    especificidade = np.zeros([num_classes])
    confiabilidade_positiva = np.zeros([num_classes])
    confiabilidade_negativa = np.zeros([num_classes])
    
    tp = np.zeros([num_classes])
    tn = np.zeros([num_classes])
    fn = np.zeros([num_classes])
    fp = np.zeros([num_classes])
    
    acertos = 0
    acuracia = 0
    total = 0

    for classe in range(num_classes):
        for linha in range(num_classes):
            for coluna in range(num_classes):
                if classe == linha == coluna:
                    tp[classe] += matriz[linha][coluna]
                elif classe != linha == coluna:
                    tn[classe] += matriz[linha][coluna]
                elif classe == linha != coluna:
                    fn[classe] += matriz[linha][coluna]
                elif classe == coluna != linha:
                    fp[classe] += matriz[linha][coluna]

    if verbose:
        print('TP =', tp)
        print('TN =', tn)
        print('FN =', fn)
        print('FP =', fp)

    for linha in range(num_classes):
        for coluna in range(num_classes):
            if linha == coluna:
                acertos += matriz[linha][coluna]
            total += matriz[linha][coluna]

    acuracia = (acertos*100)/total

    for classe in range(num_classes):
        sensibilidade[classe] = tp[classe]/(tp[classe] + fn[classe])
        especificidade[classe] = tn[classe]/(tn[classe] + fp[classe])
        confiabilidade_positiva[classe] = tp[classe]/(tp[classe] + fp[classe])
        confiabilidade_negativa[classe] = tn[classe]/(tn[classe] + fn[classe])
        if tp[classe] + fn[classe] == 0:
            sensibilidade[classe] = 0
        if tn[classe] + fp[classe] == 0:
            especificidade[classe] = 0
        if tp[classe] + fp[classe] == 0:
            confiabilidade_positiva[classe] = 0
        if tn[classe] + fn[classe] == 0:
            confiabilidade_negativa[classe] = 0
        
        if verbose:
            print('----------- Classe %d -----------' %(classe+1))
            print('Sensibilidade: ', sensibilidade[classe])
            print('Especificidade: ', especificidade[classe])
            print('Confiabilidade Positiva: ', confiabilidade_positiva[classe])
            print('Confiabilidade Negativa: ', confiabilidade_negativa[classe])
    if verbose:        
        print('Media da Sensibilidade: ', np.median(sensibilidade))
        print('Media da Especificidade: ', np.median(especificidade))
        print('Media da Confiabilidade Positiva: ', np.median(confiabilidade_positiva))
        print('Media da Confiabilidade Negativa: ', np.median(confiabilidade_negativa))

    print('Acurácia:', acuracia, end='')
    
    return acuracia
    

# Tratamento do Banco de Dados

In [ ]:
def tratar_bd(banco, coluna):
    tipos_saidas = banco[coluna].unique()
    
    base_treino = np.array([])
    base_teste = np.array([])
    
    tamanho_treino = 0
    tamanho_teste = 0
    
    for classe in tipos_saidas:
        banco_auxiliar = banco.query('%s==%d' % (coluna, classe))
        treino_auxiliar = retorna_treino(banco_auxiliar)
        
        for linha in (treino_auxiliar.values):
            base_treino = np.concatenate((base_treino, linha), axis=0)
            
        teste_auxiliar = retorna_teste(banco_auxiliar, treino_auxiliar)
        
        for linha in (teste_auxiliar.values):
            base_teste = np.concatenate((base_teste, linha), axis=0)
            
        tamanho_treino += len(treino_auxiliar)
        tamanho_teste += len(teste_auxiliar)
        
    num_colunas = len(banco.columns)
    
    base_treino = np.reshape(base_treino, (tamanho_treino, num_colunas))
    base_teste = np.reshape(base_teste, (tamanho_teste, num_colunas))
    base_treino = pd.DataFrame(base_treino, columns=banco.columns)
    base_teste = pd.DataFrame(base_teste, columns=banco.columns)

    return base_treino, base_teste, tipos_saidas
    

# Classes de Nós

In [ ]:
class No(object):
    def __init__(self, atributo=None, entropia=None, pergunta=None, filho_esquerdo=None, filho_direito=None):
        self.atributo = atributo
        self.entropia = entropia
        self.pergunta = pergunta
        self.filho_esquerdo = filho_esquerdo
        self.filho_direito = filho_direito
    
    def __repr__(self):
        return '{} - {} - {}'.format(self.atributo, self.entropia, self.pergunta)


class Folha(object):
    def __init__(self, banco, alvo):
        self.banco = banco
        self.alvo = alvo
        self.filho_esquerdo = None
        self.filho_direito = None
        self.classe = st.mode(self.banco[self.alvo])
        

# Classe Pergunta

In [ ]:
class Pergunta(object):

    def __init__(self, coluna, valor):
        self.coluna = coluna
        self.valor = valor

    def is_numeric(self, valor):
        return isinstance(valor, int) or isinstance(valor, float)

    def verifica(self, exemplo):
        valor = exemplo[self.coluna]

        if self.is_numeric(valor):
            return valor >= self.valor
        else:
            return valor == self.valor

    def __repr__(self):
        condicao = "=="
        if self.is_numeric(self.valor):
            condicao= ">="
        return "%s %s %s?" % (self.coluna, condicao, str(self.valor))


# Árvore de Decisão

In [ ]:
class ArvoreDecisao(object):
        
    def __init__(self, banco_de_dados, coluna_alvo):
        
        self.banco_de_dados = banco_de_dados

        self.coluna_alvo = coluna_alvo
        self.dados_alvo = self.banco_de_dados[coluna_alvo]

        self.n_linhas = banco_de_dados.shape[0]
        self.n_colunas = banco_de_dados.shape[1]
        self.colunas = banco_de_dados.columns

        self.raiz = None
        
        self.cria_arvore()

    def __repr__(self):
        return 'Linhas:{}\nColunas:{}'.format(self.n_linhas, self.n_colunas)
    
    def altura(self):
        return self._altura(self.raiz, 0)
        
    def _altura(self, no_atual, altura_atual):
        if not no_atual:
            return altura_atual
        altura_esq = self._altura(no_atual.filho_esquerdo, altura_atual + 1)
        altura_dir = self._altura(no_atual.filho_direito, altura_atual + 1)
        return max(altura_esq, altura_dir)

    def cria_arvore(self):
        self.raiz = self.cria_arvore_recursiva(banco=self.banco_de_dados)

    def cria_arvore_recursiva(self, banco):
        
        no = self.verifica_melhor_corte(banco)

        n_ocorrencias_classe_esquerda = list(Counter(no.filho_esquerdo[self.coluna_alvo]).values())
        
        n_ocorrencias_classe_direita = list(Counter(no.filho_direito[self.coluna_alvo]).values())
        
        '''
        CONDIÇÃO DE PARADA
        '''
        if len(n_ocorrencias_classe_esquerda) != 1:
            no.filho_esquerdo = self.cria_arvore_recursiva(no.filho_esquerdo)
        else:
            no.filho_esquerdo = Folha(no.filho_esquerdo, self.coluna_alvo)

        if len(n_ocorrencias_classe_direita) != 1:
            no.filho_direito = self.cria_arvore_recursiva(no.filho_direito)
        else:
            no.filho_direito = Folha(no.filho_direito, self.coluna_alvo)
            
        return no

    def verifica_melhor_corte(self, banco):
        maior_ganho = None
        filho_esquerdo = None
        filho_direito = None
        
        entropia_pai = self.calcula_entropia(banco)
        
        for coluna in self.colunas[1:]:
            banco_coluna = self.arredonda_float(banco[coluna])

            for linha in banco_coluna[:-1]:
                pergunta = Pergunta(coluna, linha)
                banco_esquerdo, banco_direito = self.corta_banco(banco, pergunta)
                ganho_info = self.calcula_ganho_informacao(banco_esquerdo, banco_direito, entropia_pai)
                if ganho_info > maior_ganho[1]:
                    maior_ganho = {'Atributo': coluna, 'Ganho de Informação': ganho_info, 'Pergunta': pergunta}
                    filho_esquerdo, filho_direito = banco_esquerdo, banco_direito
                    
        return No(atributo=maior_ganho['Atributo'],
                  entropia=entropia_pai,
                  pergunta=maior_ganho['Pergunta'],
                  filho_esquerdo=filho_esquerdo,
                  filho_direito=filho_direito)

    def calcula_entropia(self, banco):
        
        entropia = 0
        
        numero_ocorrencias_classe = list(Counter(banco[self.coluna_alvo]).values())

        n_linhas = banco.shape[0]
        
        for quantidade in numero_ocorrencias_classe:
            peso = (quantidade / n_linhas)
            entropia += - peso * math.log(peso, 2)
            
        return entropia

    def calcula_ganho_informacao(self, banco_esquerdo, banco_direito, entropia_pai):
        
        ganho_informacao = 0
        
        n_amostras = banco_esquerdo.shape[0] + banco_direito.shape[0]
        peso_esquerdo = banco_esquerdo.shape[0] / n_amostras
        peso_direito = banco_direito.shape[0] / n_amostras
        
        if not banco_esquerdo.empty:
            calculo_esquerdo = peso_esquerdo * self.calcula_entropia(banco_esquerdo)
        else:
            calculo_esquerdo = 0
            
        calculo_direito = peso_direito * self.calcula_entropia(banco_direito)
        
        ganho_informacao = calculo_esquerdo + calculo_direito
        
        return entropia_pai - ganho_informacao

    def corta_banco(self, banco, pergunta):
        
        linha_true, linha_false = [], []
        
        for _, row in banco.iterrows():

            if pergunta.verifica(row):
                linha_true.append(row)

            else:
                linha_false.append(row)
        
        banco_esquerdo = pd.DataFrame(linha_false)
        banco_direito = pd.DataFrame(linha_true)
        
        return banco_esquerdo, banco_direito

    def arredonda_float(self, banco):
        return list(map(int, Counter(['%d' % elem for elem in list(Counter(banco).keys())])))
    
    def percorre_arvore(self, linha, no_pai):

        if type(no_pai) == Folha:
            return no_pai.classe

        else:
            indice_pergunta = list(self.banco_de_dados.columns).index(no_pai.pergunta.coluna)

            if linha[indice_pergunta] >= no_pai.pergunta.valor:
                return self.percorre_arvore(linha, no_pai.filho_direito)

            else:
                return self.percorre_arvore(linha, no_pai.filho_esquerdo)

    def classifica(self, banco):

        serie_predicao = []

        for index, linha in banco.iterrows():
            classe = self.percorre_arvore(linha, self.raiz)
            serie_predicao.append(classe)

        serie_predicao = pd.Series(serie_predicao, name='predicao')
        predicao = pd.concat([banco[self.coluna_alvo], serie_predicao], axis=1)

        return predicao

# Execução da Árvore

In [ ]:
banco = bd_vinho
coluna_alvo = 'Vinho'

acuracias = {'Base Teste':[],'Base Treino':[], 'Base Total':[]}

for i in range(10):
    
    base_treino, base_teste, tipos_saidas = tratar_bd(banco, coluna_alvo)
    dict_bancos = {'Base Teste':base_teste,'Base Treino':base_treino, 'Base Total':banco}
    
    ad = ArvoreDecisao(base_treino, coluna_alvo)
    
    for nome_banco in list(dict_bancos.keys()):

        predicao = ad.classifica(dict_bancos[nome_banco])
        matriz_confusao = np.zeros((3, 3))

        soma = 0

        for _, linha in predicao.iterrows():
            valor_predicao = int(linha[1]) - 1
            valor_real = int(linha[0]) - 1

            matriz_confusao[valor_predicao][valor_real] += 1

        acuracias[nome_banco].append(calcula_resultados(matriz_confusao))

acuracia_bdteste = ['Desvio Padrão das Acurácias da base de teste:', np.std(acuracias['Base Teste']),
                    '| Média das Acurácias da base de teste:', np.mean(acuracias['Base Teste'])]
acuracia_bdtreino = ['Desvio Padrão das Acurácias da base de treino:', np.std(acuracias['Base Treino']),
                     '| Média das Acurácias da base de treino:', np.mean(acuracias['Base Treino'])]
acuracia_bdtotal = ['Desvio Padrão das Acurácias da base total:'np.std(acuracias['Base Total']), 
                    '| Média das Acurácias da base total:', np.mean(acuracias['Base Total'])]

print(acuracia_bdteste)
print(acuracia_bdtreino)
print(acuracia_bdtotal)
